In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install niftynet
!pip install simpleITK
!pip install nibabel

In [0]:
def load_nifty_volume_as_array(filename):
    img = nibabel.load(filename)
    data = img.get_data()
    data = np.transpose(data, [2,1,0])
    return data

In [0]:
def sensitivity(ground_true, predicted):
    P = np.sum(ground_true)
    TP = np.sum(ground_true * predicted)
    return TP / P

In [0]:
def specificity(ground_true, predicted):
    N = np.prod(ground_true.shape) - np.sum(ground_true)
    TN = np.sum(np.logical_not(ground_true) * np.logical_not(predicted))
    return TN / N

In [0]:
def binary_dice3d(s,g):
    assert(len(s.shape)==3)
    [Ds, Hs, Ws] = s.shape
    [Dg, Hg, Wg] = g.shape
    assert(Ds==Dg and Hs==Hg and Ws==Wg)
    prod = np.multiply(s, g)
    s0 = prod.sum()
    s1 = s.sum()
    s2 = g.sum()
    dice = (2.0*s0 + 1e-10)/(s1 + s2 + 1e-10)
    return dice

In [0]:
def haussdorff(ground_true, predicted):
    u = np.array(np.where(ground_true)).T
    v = np.array(np.where(predicted)).T
    hd, _, _ = scipy.spatial.distance.directed_hausdorff(u, v)
    return hd

In [0]:
from __future__ import absolute_import, print_function
import os
import sys
sys.path.append('./')
import numpy as np
import nibabel
import math
import scipy.spatial

def get_ground_truth_names(g_folder, patient_names_file):
    with open(patient_names_file) as f:
            content = f.readlines()
            patient_names = [x.strip() for x in content]
    full_gt_names = []
    for patient_name in patient_names:
        patient_dir = os.path.join(g_folder, patient_name)
        img_names   = os.listdir(patient_dir)
        gt_name = None
        for img_name in img_names:
            if 'seg.' in img_name:
                gt_name = img_name
                break
        gt_name = os.path.join(patient_dir, gt_name)
        full_gt_names.append(gt_name)
    return full_gt_names

def get_segmentation_names(seg_folder, patient_names_file):
    with open(patient_names_file) as f:
            content = f.readlines()
            patient_names = [x.strip() for x in content]
    full_seg_names = []
    for patient_name in patient_names:
        seg_name = os.path.join(seg_folder, patient_name + '.nii.gz')
        full_seg_names.append(seg_name)
    return full_seg_names

def dice_of_brats_data_set(gt_names, seg_names, type_idx):
    assert(len(gt_names) == len(seg_names))
    dice_all_data = []
    for i in range(len(gt_names)):
        g_volume = load_nifty_volume_as_array(gt_names[i])
        s_volume = load_nifty_volume_as_array(seg_names[i])
        dice_one_volume = []
        if(type_idx ==0): # whole tumor
            temp_dice = binary_dice3d(s_volume > 0, g_volume > 0)
            dice_one_volume = [temp_dice]
        elif(type_idx == 1): # tumor core
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_dice = binary_dice3d(s_volume > 0, g_volume > 0)
            dice_one_volume = [temp_dice]
        else:
            s_volume[s_volume == 1] = 0
            g_volume[g_volume == 1] = 0
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_dice = binary_dice3d(s_volume > 0, g_volume > 0)
            dice_one_volume.append(temp_dice)
        dice_all_data.append(dice_one_volume)
    return dice_all_data

def haussdorff_of_brats_data_set(gt_names, seg_names, type_idx):
    assert(len(gt_names) == len(seg_names))
    haussdorff_all_data = []
    for i in range(len(gt_names)):
        g_volume = load_nifty_volume_as_array(gt_names[i])
        s_volume = load_nifty_volume_as_array(seg_names[i])
        haussdorff_one_volume = []
        if(type_idx ==0): # whole tumor
            temp_hauss = haussdorff(s_volume > 0, g_volume > 0)
            haussdorff_one_volume = [temp_hauss]
        elif(type_idx == 1): # tumor core
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_hauss = haussdorff(s_volume > 0, g_volume > 0)
            haussdorff_one_volume = [temp_hauss]
        else:
            s_volume[s_volume == 1] = 0
            g_volume[g_volume == 1] = 0
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_hauss = haussdorff(s_volume > 0, g_volume > 0)
            haussdorff_one_volume.append(temp_hauss)
        haussdorff_all_data.append(haussdorff_one_volume)
    return haussdorff_all_data
def sensitivity_of_brats_data_set(gt_names, seg_names, type_idx):
    assert(len(gt_names) == len(seg_names))
    sens_all_data = []
    for i in range(len(gt_names)):
        g_volume = load_nifty_volume_as_array(gt_names[i])
        s_volume = load_nifty_volume_as_array(seg_names[i])
        sens_one_volume = []
        if(type_idx ==0): # whole tumor
            temp_sens = sensitivity(s_volume > 0, g_volume > 0)
            if math.isnan(temp_sens):
              temp_sens = 0
            sens_one_volume = [temp_sens]
        elif(type_idx == 1): # tumor core
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_sens = sensitivity(s_volume > 0, g_volume > 0)
            if math.isnan(temp_sens):
              temp_sens = 0
            sens_one_volume = [temp_sens]
        else:
            s_volume[s_volume == 1] = 0
            g_volume[g_volume == 1] = 0
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_sens = sensitivity(s_volume > 0, g_volume > 0)
            if math.isnan(temp_sens):
              temp_sens = 0
            sens_one_volume.append(temp_sens)
        sens_all_data.append(sens_one_volume)
    return sens_all_data
def specificity_of_brats_data_set(gt_names, seg_names, type_idx):
    assert(len(gt_names) == len(seg_names))
    spec_all_data = []
    for i in range(len(gt_names)):
        g_volume = load_nifty_volume_as_array(gt_names[i])
        s_volume = load_nifty_volume_as_array(seg_names[i])
        spec_one_volume = []
        if(type_idx ==0): # whole tumor
            temp_spec = specificity(s_volume > 0, g_volume > 0)
            if not math.isnan(temp_spec):
              spec_one_volume = [temp_spec]
        elif(type_idx == 1): # tumor core
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_spec = specificity(s_volume > 0, g_volume > 0)
            if not math.isnan(temp_spec):
              spec_one_volume = [temp_spec]
        else:
            s_volume[s_volume == 1] = 0
            g_volume[g_volume == 1] = 0
            s_volume[s_volume == 2] = 0
            g_volume[g_volume == 2] = 0
            temp_spec = specificity(s_volume > 0, g_volume > 0)
            if not math.isnan(temp_spec):
              spec_one_volume.append(temp_spec)
        spec_all_data.append(spec_one_volume)
    return spec_all_data

In [29]:
if __name__ == '__main__':
    s_folder = 'drive/app/BRATS2017/resultts2017/'
    g_folder = 'drive/app/BRATS2017/'
    patient_names_file = 'drive/app/BRATS2017/Data_names/test_names.txt'
    patient_names_file1 = 'drive/app/BRATS2017/Data_names/test_names1.txt'
    test_types = ['whole','core', 'enhac']
    gt_names  = get_ground_truth_names(g_folder, patient_names_file1)
    seg_names = get_segmentation_names(s_folder, patient_names_file)
    for type_idx in range(3):
        dice = dice_of_brats_data_set(gt_names, seg_names, type_idx)
        hauss = haussdorff_of_brats_data_set(gt_names, seg_names, type_idx)
        sens = sensitivity_of_brats_data_set(gt_names, seg_names, type_idx)
        spec = specificity_of_brats_data_set(gt_names, seg_names, type_idx)
        dice = np.asarray(dice)
        dice_mean = dice.mean(axis = 0)
        dice_std  = dice.std(axis  = 0)
        test_type = test_types[type_idx]
        np.savetxt(s_folder + '/dice_{0:}.txt'.format(test_type), dice)
        np.savetxt(s_folder + '/dice_{0:}_mean.txt'.format(test_type), dice_mean)
        np.savetxt(s_folder + '/dice_{0:}_std.txt'.format(test_type), dice_std)
        print('tissue type', test_type)
        print('dice mean  ', dice_mean)
        print('dice std   ', dice_std)
        print('\n')
        hauss = np.asarray(hauss)
        hauss_mean = hauss.mean(axis = 0)
        hauss_std  = hauss.std(axis  = 0)
        np.savetxt(s_folder + '/hauss_{0:}.txt'.format(test_type), hauss)
        np.savetxt(s_folder + '/hauss_{0:}_mean.txt'.format(test_type), hauss_mean)
        np.savetxt(s_folder + '/hauss_{0:}_std.txt'.format(test_type), hauss_std)
        print('hauss mean  ', hauss_mean)
        print('hauss std   ', hauss_std)
        print('\n')
        sens = np.asarray(sens)
        sens_mean = sens.mean(axis = 0)
        sens_std  = sens.std(axis  = 0)
        np.savetxt(s_folder + '/sens_{0:}.txt'.format(test_type), sens)
        np.savetxt(s_folder + '/sens_{0:}_mean.txt'.format(test_type), sens_mean)
        np.savetxt(s_folder + '/sens_{0:}_std.txt'.format(test_type), sens_std)
        print('sens mean  ', sens_mean)
        print('sens std   ', sens_std)
        print('\n')
        spec = np.asarray(spec)
        spec_mean = spec.mean(axis = 0)
        spec_std  = spec.std(axis  = 0)
        np.savetxt(s_folder + '/spec_{0:}.txt'.format(test_type), spec)
        np.savetxt(s_folder + '/spec_{0:}_mean.txt'.format(test_type), spec_mean)
        np.savetxt(s_folder + '/spec_{0:}_std.txt'.format(test_type), spec_std)
        print('spec mean  ', spec_mean)
        print('spec std   ', spec_std)
        print('\n')
        print('\n')

tissue type whole
dice mean   [0.9048875]
dice std    [0.06350397]


hauss mean   [8.44506981]
hauss std    [8.75775883]


sens mean   [0.90256649]
sens std    [0.0767025]


spec mean   [0.99926776]
spec std    [0.00064345]




tissue type core
dice mean   [0.91404211]
dice std    [0.04785603]


hauss mean   [4.72256245]
hauss std    [3.35279936]


sens mean   [0.91305848]
sens std    [0.05725224]


spec mean   [0.99968673]
spec std    [0.00028089]






/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


tissue type enhac
dice mean   [0.78412107]
dice std    [0.25146548]


hauss mean   [4.74275159]
hauss std    [5.63177003]


sens mean   [0.75421421]
sens std    [0.29468842]


spec mean   [0.99972842]
spec std    [0.00046265]






In [27]:
sens

array([list([0.7417721518987341]), list([0.7918821096173733]),
       list([0.6543586585297576]), list([0.7342857142857143]),
       list([0.732127794179671]), list([0.8802968305707747]),
       list([0.9139778509428315]), list([0.8385170440408062]),
       list([0.9825912503235827]), list([0.8622678616211079]),
       list([0.9031523486336845]), list([0.8585911189806618]),
       list([0.9525306928043354]), list([0.8641286800317004]),
       list([0.9847044427710844]), list([0.9387280998935243]),
       list([0.7246436686634993]), list([0.9554940374787053]),
       list([0.8783625730994152]), list([0.8583921964345779]),
       list([0.9472202823101179]), list([0.7346436266103856]),
       list([0.8906028469347247]), list([0.8499237223493517]),
       list([0.8073610882562668]), list([0.8296173390023851]),
       list([0.9129981118950611]), list([0.6851352490098592]),
       list([0.7594988424349721]), list([0.8623230560403335]),
       list([0.9815586149883947]), list([0.9957732815964